In [30]:
import re 
import string
import timestring
from sklearn.linear_model import LinearRegression, LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_score, KFold
import glob, os
import numpy as np
from sklearn import svm

In [42]:
#DIR = 'C:/Users/Elaine/Desktop/TCC/all timeline/'
import os, io, json, codecs

os.chdir(DIR)
date_ative = []
countuser = 0
user = []
positive_sample=[]
negative_sample = []

#for file in glob.glob("*.txt"):
 #   print 'file ', file, '\n'
with io.open(DIR+'DaviWesler.txt.txt','r',encoding="utf-8") as f:#, codecs.open("CleuzaBapti - Copy.txt", 'w+', encoding='utf-8') as fi:
    lines = f.readlines()
    i=0
    for i in xrange(len(lines)):
        tweet= json.loads(lines[i])        
        if 'foraDILMA' in tweet['text'] or 'foradilma' in tweet['text'] or 'Fora Dilma' in tweet['text'] or 'foraDilma' in tweet['text'] or 'FORADILMA' in tweet['text'] or 'fora Dilma' in tweet['text'] or 'FORA DILMA' in tweet['text'] or 'ForaDilma' in tweet['text'] or'foraPT' in tweet['text'] or 'FORAPT' in tweet['text'] or 'fora pt' in tweet['text']or 'FORA PT' in tweet['text'] or 'ForaPT' in tweet['text'] or'vempraRua' in tweet['text'] or 'vemprarua' in tweet['text'] or 'vem pra rua' in tweet['text']or 'VEMPRARUA' in tweet['text'] or 'VemPraRua' in tweet['text']:
            print tweet['created_at']
            j=i-1
            while j>1:
                #print j
                tweet1= json.loads(lines[j])    
                #print tweet['time_tweet']
                positive_sample.append(tweet1)
                j=j-1
                if j == 1:
                    print 'start of the file'
                    break
            negative_sample = positive_sample[10:]
            break


Sun Aug 16 16:41:54 +0000 2015
start of the file


In [43]:
for p in positive_sample:
    print p['text']
    break

RT @curiosiddades: Timão e Pumba, são vcs? Esse filhote de javali e esse suricato viraram grandes amigos em zoológico da Inglaterra. http:/…


In [44]:
vectorizer = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, binary=False, norm='l2')
X = vectorizer.fit_transform(t['text'] for t in positive_sample)
print('Vectorized %d tweets. Found %d terms.' % (X.shape[0], X.shape[1]))
features = np.array(vectorizer.get_feature_names())

Vectorized 236 tweets. Found 403 terms.


In [49]:
features


array([u'100', u'104', u'104 anos', u'11', u'2015', u'29', u'84',
       u'84 anos', u'aben\xe7oe', u'acesse', u'acesse http', u'acho',
       u'acho que', u'acidente', u'acidente de', u'acompanhe',
       u'acompanhe http', u'acordei', u'adbrasil', u'afffmanu', u'agora',
       u'ainda', u'alagoas', u'alerta', u'amigo', u'amigos', u'amor',
       u'ano', u'ano no', u'anos', u'anos http', u'anos que', u'anuncia',
       u'anunciai_', u'ao', u'ao seu', u'ao vivo', u'aos', u'apenas',
       u'app', u'ap\xf3s', u'ap\xf3s acidente', u'aqui', u'ara\xfajo',
       u'ara\xfajo morre', u'argentina', u'as', u'as pessoas',
       u'assembleia', u'assembleia de', u'assembl\xe9ia',
       u'assembl\xe9ia de', u'assim', u'assista', u'assista ao', u'at\xe9',
       u'aula', u'banda', u'bar\xe7a', u'bar\xe7a championsleaguefinal',
       u'bem', u'boa', u'boa noite', u'bom', u'bom dia', u'brasil',
       u'brasileira', u'brincadero', u'cada', u'campe\xf5es', u'cantor',
       u'carro', u'casa', u'cei

In [41]:
with codecs.open("C:/Users/Elaine/"+'file_test', 'w+', encoding='utf-8') as fi:
       for f in features:
            fi.write(f)

In [381]:
def tokenize(text):
    punc_re = '[' + '\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\_\\{\\|\\}' + ']'
    #print punc_re
    text = text.lower()
    text = re.sub('#(\S+)', r'HASHTAG_\1', text)
    text = re.sub('@(\S+)', r'MENTION_\1', text)
    text = re.sub('http\S+', 'THIS_IS_A_URL', text)
    text = re.sub(r'(.)\1\1\1+', r'\1', text)
    text = re.sub(r'[0-9]', '9', text)
    toks = []
    for tok in text.split():
        tok = re.sub(r'^(' + punc_re + '+)', r'\1 ', tok)
        #print '1',tok
        tok = re.sub(r'(' + punc_re + '+)$', r' \1', tok)
        #print '2',tok

        for subtok in tok.split():
            #print '3',subtok
            if re.search('\w', subtok):
                toks.append(subtok)
    return toks